<a href="https://colab.research.google.com/github/prof-nuduls/EE628HW/blob/main/AAI627HW8Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz

In [3]:
!tar xf spark-3.5.1-bin-hadoop3.tgz

In [4]:
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [5]:
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [57]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

from google.colab import files

In [76]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [77]:
training = spark.read.csv("trainItem.data", header = False)
training.show(5)

+------+------+---+
|   _c0|   _c1|_c2|
+------+------+---+
|199808|248969| 90|
|199808|  2663| 90|
|199808| 28341| 90|
|199808| 42563| 90|
|199808| 59092| 90|
+------+------+---+
only showing top 5 rows



In [78]:
training = training.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "rating")
training.show(5)

+------+------+------+
|userID|itemID|rating|
+------+------+------+
|199808|248969|    90|
|199808|  2663|    90|
|199808| 28341|    90|
|199808| 42563|    90|
|199808| 59092|    90|
+------+------+------+
only showing top 5 rows



In [79]:
from pyspark.sql.types import IntegerType
training = training.withColumn("userID", training["userID"].cast(IntegerType()))
training = training.withColumn("itemID", training["itemID"].cast(IntegerType()))
training = training.withColumn("rating", training["rating"].cast('float'))
training.show(3)

+------+------+------+
|userID|itemID|rating|
+------+------+------+
|199808|248969|  90.0|
|199808|  2663|  90.0|
|199808| 28341|  90.0|
+------+------+------+
only showing top 3 rows



In [80]:
# Create ALS model
als = ALS(
    maxIter=20,
    rank = 20,
    regParam=0.01,
    userCol="userID",
    itemCol="itemID",
    ratingCol="rating",
    nonnegative = True,
    implicitPrefs = False,
    coldStartStrategy="drop"
)

In [81]:
model = als.fit(training)

In [82]:
testing = spark.read.csv("testItem.data", header = False)

In [83]:
testing = testing.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "rating")
testing.show(5)

+------+------+------+
|userID|itemID|rating|
+------+------+------+
|199810|208019|     0|
|199810| 74139|     0|
|199810|  9903|     0|
|199810|242681|     0|
|199810| 18515|     0|
+------+------+------+
only showing top 5 rows



In [84]:
testing = testing.withColumn("userID", testing["userID"].cast(IntegerType()))
testing = testing.withColumn("itemID", testing["itemID"].cast(IntegerType()))
testing = testing.withColumn("rating", testing["rating"].cast('float'))
testing.show(3)


+------+------+------+
|userID|itemID|rating|
+------+------+------+
|199810|208019|   0.0|
|199810| 74139|   0.0|
|199810|  9903|   0.0|
+------+------+------+
only showing top 3 rows



In [85]:
testing.count()

120000

In [86]:
predictions = model.transform(testing)
predictions.show(5)


+------+------+------+----------+
|userID|itemID|rating|prediction|
+------+------+------+----------+
|233686|     1|   0.0|  40.29612|
|215400|     3|   0.0| 48.374557|
|224379|     5|   0.0| 90.574135|
|200179|    13|   0.0| 63.886276|
|199859|    17|   0.0| 60.136387|
+------+------+------+----------+
only showing top 5 rows



In [87]:
predictions.count()

119974

In [88]:
from pyspark.sql.functions import lit

missing_combinations = testing.join(predictions, ["userID", "itemID"], "left_anti")
missing_combinations = missing_combinations.withColumn("prediction", lit(0))

missing_combinations.show()

+------+------+------+----------+
|userID|itemID|rating|prediction|
+------+------+------+----------+
|233600|244986|   0.0|         0|
|240579| 12238|   0.0|         0|
|201764| 99699|   0.0|         0|
|245355| 19630|   0.0|         0|
|244261| 17777|   0.0|         0|
|217814|282196|   0.0|         0|
|209427|182926|   0.0|         0|
|239834| 75679|   0.0|         0|
|242951| 83528|   0.0|         0|
|234512|179776|   0.0|         0|
|245335| 79059|   0.0|         0|
|228550|204660|   0.0|         0|
|236927|198558|   0.0|         0|
|244356|237110|   0.0|         0|
|234911|175541|   0.0|         0|
|205578| 40556|   0.0|         0|
|239626|182926|   0.0|         0|
|240299|128682|   0.0|         0|
|227794|235968|   0.0|         0|
|211542|236983|   0.0|         0|
+------+------+------+----------+
only showing top 20 rows



In [89]:
final_predictions = predictions.union(missing_combinations)
final_predictions.show(5)

+------+------+------+----------+
|userID|itemID|rating|prediction|
+------+------+------+----------+
|233686|     1|   0.0|  40.29612|
|215400|     3|   0.0| 48.374557|
|224379|     5|   0.0| 90.574135|
|200179|    13|   0.0| 63.886276|
|199859|    17|   0.0| 60.136387|
+------+------+------+----------+
only showing top 5 rows



In [90]:
from pyspark.sql import Window,functions
from pyspark.sql.functions import col, rank, when, row_number,concat

# Define a window partitioned by userID, ordered by prediction in ascending order
window_spec = Window.partitionBy("userID").orderBy("prediction")

# Assign suggestion values based on the bottom three predictions within each user ID group
final_predictions = final_predictions.withColumn("suggestion", when(row_number().over(window_spec) > 3, 1).otherwise(0))

final_predictions.show()

selected_predictions = final_predictions.select("userID", "itemID", "suggestion")
selected_predictions.show()

+------+------+------+----------+----------+
|userID|itemID|rating|prediction|suggestion|
+------+------+------+----------+----------+
|199810|242681|   0.0| 37.038292|         0|
|199810| 18515|   0.0| 51.204964|         0|
|199810| 74139|   0.0| 57.724483|         0|
|199810|208019|   0.0|   58.7052|         1|
|199810|  9903|   0.0|  77.97268|         1|
|199810|105760|   0.0|  146.2458|         1|
|199812|276940|   0.0| 67.834206|         0|
|199812|130023|   0.0|  75.72296|         0|
|199812| 29189|   0.0|  80.84672|         0|
|199812|223706|   0.0|  94.32001|         1|
|199812|142408|   0.0|  107.6084|         1|
|199812|211361|   0.0| 145.04396|         1|
|199813|111874|   0.0| 33.612457|         0|
|199813| 20968|   0.0|  48.51801|         0|
|199813|188441|   0.0|  55.37331|         0|
|199813|184173|   0.0|  82.19245|         1|
|199813| 79640|   0.0|112.806915|         1|
|199813| 21571|   0.0| 324.22968|         1|
|199814|122429|   0.0| 14.542861|         0|
|199814|26

In [91]:
# ensure equal number of 0 and 1s
num_zeros = final_predictions.filter(functions.col("suggestion") == 0).count()

print("Number of 0s in 'suggestion' column:", num_zeros)

Number of 0s in 'suggestion' column: 60000


In [92]:
# Concatenate 'userID' and 'itemID' columns to create 'UserID_TrackID' column
selected_predictions = selected_predictions.withColumn('UserID_TrackID', concat(col('userID').cast('string'),lit('_'), col('itemID').cast('string')))

# Select 'UserID_TrackID' and 'suggestion' columns
df = selected_predictions.select('UserID_TrackID', 'suggestion')

# Rename columns to match the desired output
df = df.withColumnRenamed('UserID_TrackID', 'TrackID').withColumnRenamed('suggestion', 'Predictor')
df.show()



+-------------+---------+
|      TrackID|Predictor|
+-------------+---------+
|199810_242681|        0|
| 199810_18515|        0|
| 199810_74139|        0|
|199810_208019|        1|
|  199810_9903|        1|
|199810_105760|        1|
|199812_276940|        0|
|199812_130023|        0|
| 199812_29189|        0|
|199812_223706|        1|
|199812_142408|        1|
|199812_211361|        1|
|199813_111874|        0|
| 199813_20968|        0|
|199813_188441|        0|
|199813_184173|        1|
| 199813_79640|        1|
| 199813_21571|        1|
|199814_122429|        0|
|199814_262193|        0|
+-------------+---------+
only showing top 20 rows



In [93]:
df.coalesce(1).write.mode('overwrite').csv("output")
df.toPandas().to_csv('output1.csv',index=False)